In [55]:
import pandas as pd

routes = pd.read_csv('routes_raw_data.csv')

In [56]:
routes.head()

,Порядковый номер,Идентификатор маршрута,Числовой номер маршрута,Наименование маршрута,Тип транспорта,Направление движения,ID остановки,ID следующей остановки,"Расстояние до следующей остановки, км",Название остановки,Координаты остановки
0,1,1125,А,Река Оккервиль - Ул. Коллонтай,Трамвай,1,16396,16367,0.80,"Конечная станция ""Река Оккервиль""","59.908880895247805,30.4946709"
1,2,1125,А,Река Оккервиль - Ул. Коллонтай,Трамвай,1,16367,16371,0.40,Пр. Солидарности,"59.911856650308394,30.4998817"
2,3,1125,А,Река Оккервиль - Ул. Коллонтай,Трамвай,1,16371,16398,0.20,"Пр. Солидарности, 19","59.915934610913702,30.4975796"
3,4,1125,А,Река Оккервиль - Ул. Коллонтай,Трамвай,1,16398,16400,0.40,Ул. Подвойского,"59.917403375763719,30.496191"
4,5,1125,А,Река Оккервиль - Ул. Коллонтай,Трамвай,1,16400,16401,0.45,Торговый центр,"59.921340303564129,30.4925327"


In [57]:
tram_routes = routes[routes['Тип транспорта'] == 'Трамвай']
tram_routes = tram_routes.sort_values(['Числовой номер маршрута', 'Порядковый номер'])

In [58]:
results = []
'''
for route_id, route_group in tram_routes.groupby('Идентификатор маршрута'):
    route_group = route_group.sort_values('Порядковый номер')

    for i in range(len(route_group) - 1):
        row1 = route_group.iloc[i]
        row2 = route_group.iloc[i + 1]

        results.append({
            'ID маршрута': route_id,
            'Номер маршрута': route_group['Числовой номер маршрута'].iloc[0],
            'ID1': row1['ID остановки'],
            'ID2': row2['ID остановки'],
            'Координаты1': row1['Координаты остановки'],
            'Координаты2': row2['Координаты остановки'],
        })
'''
stops_dict = {}
for i, row in tram_routes.iterrows():
    stop_id = row['ID остановки']
    coordinates = row['Координаты остановки']

    stops_dict[stop_id] = coordinates

for i, row in tram_routes.iterrows():
    current_route_id = row['Идентификатор маршрута']
    current_stop_id = row['ID остановки']
    next_stop_id = row['ID следующей остановки']

    # 4. Для каждой строки (каждой остановки) записываем её саму
    results.append({
        'ID_маршрута': current_route_id,
        'Номер_маршрута': row['Числовой номер маршрута'],
        'ID_остановки': current_stop_id,
        'Координаты': stops_dict.get(current_stop_id, 'Не найдено'),
    })

    # 5. Если есть следующая остановка, записываем и её (из словаря)
    if pd.notna(next_stop_id) and next_stop_id in stops_dict:
        results.append({
            'ID_маршрута': current_route_id,
            'Номер_маршрута': row['Числовой номер маршрута'],
            'ID_остановки': next_stop_id,
            'Координаты': stops_dict[next_stop_id],
        })

In [59]:
result_routes = pd.DataFrame(results)
result_routes.insert(0, 'Порядковый индекс', range(1, len(result_routes) + 1))

In [60]:
result_routes.head()

,Порядковый индекс,ID_маршрута,Номер_маршрута,ID_остановки,Координаты
0,1,3449,1,17003,"59.926252249824074,30.2536411"
1,2,3449,1,16493,"59.926416539291615,30.2550163"
2,3,3449,1,16493,"59.926416539291615,30.2550163"
3,4,3449,1,16498,"59.927751361058263,30.2572842"
4,5,3449,1,16498,"59.927751361058263,30.2572842"


In [61]:
result_routes.to_csv('routes_graph.csv', index=False, encoding='utf-8-sig')